In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

font = {'family':"IPAexGothic",
        'weight' : 'bold'}
mpl.rc('font', **font)

In [117]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
sample_submit = pd.read_csv("./data/sample_submit.csv")

# Data Shape

In [118]:
print("Training data:", train.shape)
print("Test data:", test.shape)

Training data: (31470, 17)
Test data: (31262, 16)


# Concat train and test
# 賃料 and ID are saved as target, total_id respectively

In [119]:
target = train["賃料"]
train = train.drop(columns=["賃料"])
total_data = pd.concat([train, test], ignore_index=True)
total_id = total_data["id"]
total_data = total_data.drop(columns="id")
total_data.shape

(62732, 15)

# Examine Missing Values

In [120]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [121]:
missing_values = missing_values_table(total_data)
missing_values

Your selected dataframe has 15 columns.
There are 9 columns that have missing values.


,Missing Values,% of Total Values
周辺環境,18673,29.8
契約期間,14101,22.5
駐車場,10643,17.0
放送・通信,6894,11.0
方角,5557,8.9
キッチン,1256,2.0
バス・トイレ,649,1.0
室内設備,155,0.2
所在階,1,0.0


# Convert 全角 to 半角

In [122]:
for column in total_data.columns:
    print(column)
    total_data[column] = total_data[column].str.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

所在地
アクセス
間取り
築年数
方角
面積
所在階
バス・トイレ
キッチン
放送・通信
室内設備
駐車場
周辺環境
建物構造
契約期間


# 築年数

## This format is "%Y年%m"
## But there are entries with "新築"

In [123]:
total_data["築年数"] = total_data["築年数"].replace("新築", "0年0ヶ月")
df = total_data["築年数"].str.split("年", expand=True)
df[0] = df[0].astype(int)
df[1] = df[1].str.strip("ヶ月").astype(int)
total_data["築年数_月"] = df[0] * 12 + df[1]
total_data.drop(columns=["築年数"], inplace=True)

# 面積

## Just converts "%dm2" to float

In [124]:
total_data["面積"] = total_data["面積"].str.replace("m2", "").astype(np.float)

# 所在階

In [125]:
# fill nan with ""-999"
total_data["所在階"] = total_data["所在階"].fillna("-999")
# add "/" to entries without "/" for splitting
total_data["所在階"][~total_data["所在階"].str.contains("/")] = "/" + total_data["所在階"][~total_data["所在階"].str.contains("/")]
# Split
total_data["階建"] = total_data["所在階"].str.split("/", expand=True)[1]
total_data["所在階"] = total_data["所在階"].str.split("/", expand=True)[0]
# Remove "階" from 所在階
total_data["所在階"] = total_data["所在階"].str.replace("階", "")
# Split again
total_data["所在階_地下"] = total_data["所在階"].str.split("地下", expand=True)[1]
total_data["所在階"] = total_data["所在階"].str.split("地下", expand=True)[0]
total_data["所在階"] = total_data["所在階"].replace("", 0).astype(int)
total_data["所在階_地下"] = total_data["所在階_地下"].fillna(0).astype(int)
total_data["階建"] = total_data["階建"].str.replace("階建", "")
total_data["階建"] = total_data["階建"].str.replace("階", "")
total_data["階建_地下"] = total_data["階建"].str.split("(", expand=True)[1]
total_data["階建"] = total_data["階建"].str.split("(", expand=True)[0].astype(int)
total_data["階建_地下"] = total_data["階建_地下"].str.replace("地下", "").str.replace(")", "").fillna(0).astype(int)

/home/amar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


# 間取り

In [126]:
total_data["納戸"] = total_data["間取り"].str.contains("納戸", regex=False)
total_data["間取り"] = total_data["間取り"].str.strip("(納戸)")

unique_keys = sorted(set(''.join(total_data["間取り"].unique())))

for char in ["D", "K", "L", "R", "S"]:
    new_col = "間取り" + "_" + char
#     print(new_col)
    total_data[new_col] = total_data["間取り"].str.contains(char)
    
total_data["間取り_数"] = total_data["間取り"].str.strip("DKLRS+").astype(int)

# 方角

In [127]:
total_data["方角_南"] = total_data["方角"].str.contains("南")
total_data["方角_東"] = total_data["方角"].str.contains("東")
total_data["方角_西"] = total_data["方角"].str.contains("西")
total_data["方角_北"] = total_data["方角"].str.contains("北")

# 建物構造

## Fine the way it is

# 契約期間

In [128]:
df = total_data["契約期間"].str.split("\t", expand=True)
total_data["契約期間_定期借家"] = total_data["契約期間"].str.split("\t", expand=True)[2].replace("定期借家", 1).fillna(0).astype(int)
total_data["契約期間"] = total_data["契約期間"].str.split("\t", expand=True)[0]
total_data["契約期間"] = total_data["契約期間"].str.replace("ヶ", "")
total_data["契約期間"] = total_data["契約期間"].fillna("0年間")
total_data["契約期間"][total_data["契約期間"].str.contains("年間", regex=False)] = (total_data["契約期間"][total_data["契約期間"].str.contains("年間", regex=False)].str.replace("年間", "").astype(int)+2019).astype(str)+"年8月まで"
total_data["契約期間"][~total_data["契約期間"].str.contains("年")] = "年" + total_data["契約期間"][~total_data["契約期間"].str.contains("年")]
temp_df = total_data["契約期間"][total_data["契約期間"].str.contains("月間", regex=False)].str.replace("月間", "").str.split("年", expand=True)
temp_df[1] = temp_df[1].astype(int) + 8
temp_df[0] = 2019 + temp_df[0].replace("", "0").astype(int)
temp_df[0][temp_df[1] > 12] = temp_df[0][temp_df[1] > 12] + 1
temp_df[1][temp_df[1] > 12] = temp_df[1][temp_df[1] > 12] - 12
total_data["契約期間"][total_data["契約期間"].str.contains("月間", regex=False)] = temp_df[0].astype(str)+ "年" + temp_df[1].astype(str) + "月まで"
total_data["契約期間"] = pd.to_datetime(total_data["契約期間"], format="%Y年%m月まで")
total_data["契約期間_月"] = (total_data["契約期間"].dt.year - 2019)*12 + total_data["契約期間"].dt.month - 8

/home/amar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/amar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/amar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


# 所在地

In [129]:
total_data["所在地"] = total_data["所在地"].str.replace("東京都", "")
total_data["所在地_区"] = total_data["所在地"].str.split("区", expand=True)[0]
total_data["所在地"] = total_data["所在地"].str.split("区", expand=True)[1]

# アクセス

In [130]:
df = total_data["アクセス"].str.split("\t", expand=True)

In [131]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,都営三田線,西巣鴨駅,徒歩4分,,埼京線,板橋駅,徒歩14分,,都電荒川線,西ケ原四丁目駅,徒歩7分,None,None
1,都営大江戸線,勝どき駅,徒歩5分,,有楽町線,月島駅,徒歩9分,,日比谷線,築地駅,徒歩20分,None,None
2,京王線,笹塚駅,徒歩6分,,京王線,代田橋駅,徒歩7分,,京王線,明大前駅,徒歩17分,None,None


## df[0]

In [132]:
# Check entries without "線"
df[0][df[0].str.contains("線") == False].unique()

array(['つくばエクスプレス', '東武スカイツリーライン', '日暮里・舎人ライナー', 'ゆりかもめ'], dtype=object)

In [133]:
total_data["アクセス_線_1"] = df[0]

## df[1]

In [134]:
# Check entries without "駅"
df[1][df[1].str.contains("駅") == False].unique()

array([], dtype=object)

In [135]:
total_data["アクセス_駅_1"] = df[1]

## df[2]

In [136]:
df[2].unique()

array(['徒歩4分', '徒歩5分', '徒歩6分', '徒歩9分', '徒歩2分', '徒歩7分', '徒歩3分', '徒歩8分',
       '徒歩15分', '徒歩11分', '徒歩10分', '徒歩13分', '徒歩12分', '徒歩22分', '徒歩1分',
       '徒歩19分', '徒歩18分', '徒歩23分', '徒歩17分', '徒歩20分', '/バス(6分)都民農園下車徒歩1分',
       '徒歩14分', '/バス(19分)長久保下車徒歩2分', '徒歩24分', '徒歩16分',
       '/バス(7分)大泉風致地区下車徒歩5分', '徒歩29分', '徒歩37分', '徒歩21分',
       '/バス(10分)関町郵便局前下車徒歩1分', '/バス(16分)新小71\u3000農業試験場下車徒歩3分',
       '/バス(18分)土支田二丁目下車徒歩5分', '/バス(12分)比丘尼橋下車徒歩3分', '/バス(10分)団地入口下車徒歩3分',
       '/バス(5分)善福寺下車徒歩1分', '徒歩25分', '/バス(15分)ふれあいの家下車',
       '/バス(13分)水元そよかぜ園下車徒歩1分', '/バス(15分)水元4丁目下車徒歩28分',
       '/バス(25分)武蔵関駅下車徒歩7分', '/バス(15分)越後山下車徒歩4分', '徒歩27分', '徒歩28分',
       '徒歩33分', '/バス(9分)西長久保下車徒歩1分', '/バス(16分)亀26\u3000松江下車徒歩5分', '徒歩26分',
       '/バス(23分)葛飾車庫入口下車徒歩11分', '/バス(10分)大田文化の森下車徒歩3分',
       '/バス(18分)幸田小学校下車徒歩3分', '/バス(11分)水元そよかぜ園下車徒歩6分',
       '/バス(16分)佐野センター前下車徒歩34分', '/バス(15分)花畑団地下車徒歩10分',
       '/バス(15分)水元そよかぜ園下車徒歩2分', '/バス(12分)六木団地下車徒歩3分',
       '/バス(19分)大泉学園町八丁目下車徒歩2分', '/バス(18分)本木新道下車徒歩3分',
     

In [137]:
df_2 = df[2].str.split("/", expand=True)
df_2.head(3)

,0,1
0,徒歩4分,None
1,徒歩5分,None
2,徒歩6分,None


In [138]:
df_2[0].unique()

array(['徒歩4分', '徒歩5分', '徒歩6分', '徒歩9分', '徒歩2分', '徒歩7分', '徒歩3分', '徒歩8分',
       '徒歩15分', '徒歩11分', '徒歩10分', '徒歩13分', '徒歩12分', '徒歩22分', '徒歩1分',
       '徒歩19分', '徒歩18分', '徒歩23分', '徒歩17分', '徒歩20分', '', '徒歩14分', '徒歩24分',
       '徒歩16分', '徒歩29分', '徒歩37分', '徒歩21分', '徒歩25分', '徒歩27分', '徒歩28分',
       '徒歩33分', '徒歩26分', '徒歩38分', '徒歩36分', '徒歩32分', '徒歩34分', '徒歩39分',
       '車3km(15分)', '徒歩30分', '徒歩31分', '徒歩42分', '徒歩40分', '車3.1km(10分)',
       '徒歩43分', '車3.9km(11分)', '徒歩49分', '徒歩35分', '徒歩44分'], dtype=object)

In [139]:
total_data.columns

Index(['所在地', 'アクセス', '間取り', '方角', '面積', '所在階', 'バス・トイレ', 'キッチン', '放送・通信',
       '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間', '築年数_月', '階建', '所在階_地下', '階建_地下',
       '納戸', '間取り_D', '間取り_K', '間取り_L', '間取り_R', '間取り_S', '間取り_数', '方角_南',
       '方角_東', '方角_西', '方角_北', '契約期間_定期借家', '契約期間_月', '所在地_区', 'アクセス_線_1',
       'アクセス_駅_1'],
      dtype='object')

In [140]:
total_data["アクセス_駅_1_徒歩_分"] = df_2[0].where(df_2[0].str.contains("徒歩", regex=False))

In [141]:
total_data["アクセス_駅_1_徒歩_分"] = total_data["アクセス_駅_1_徒歩_分"].str.strip("徒歩分").astype(float)

In [142]:
total_data["アクセス_駅_1_徒歩_分"].unique()

array([ 4.,  5.,  6.,  9.,  2.,  7.,  3.,  8., 15., 11., 10., 13., 12.,
       22.,  1., 19., 18., 23., 17., 20., nan, 14., 24., 16., 29., 37.,
       21., 25., 27., 28., 33., 26., 38., 36., 32., 34., 39., 30., 31.,
       42., 40., 43., 49., 35., 44.])

In [143]:
total_data["アクセス_駅_1_車_分"] = df_2[0].where(df_2[0].str.contains("徒歩", regex=False)==False)

In [144]:
total_data["アクセス_駅_1_車_分"] = total_data["アクセス_駅_1_車_分"].replace(r'^\s*$', np.nan, regex=True)

In [145]:
total_data["アクセス_駅_1_車_分"].unique()

array([nan, '車3km(15分)', '車3.1km(10分)', '車3.9km(11分)'], dtype=object)

In [146]:
total_data["アクセス_駅_1_車_分"] = total_data["アクセス_駅_1_車_分"].str.replace("車3km(15分)", "15", regex=False)
total_data["アクセス_駅_1_車_分"] = total_data["アクセス_駅_1_車_分"].str.replace("車3.1km(10分)", "10", regex=False)
total_data["アクセス_駅_1_車_分"] = total_data["アクセス_駅_1_車_分"].str.replace("車3.9km(11分)", "11", regex=False)
total_data["アクセス_駅_1_車_分"] = total_data["アクセス_駅_1_車_分"].astype(float)

In [147]:
total_data["アクセス_駅_1_車_分"].unique()

array([nan, 15., 10., 11.])

In [148]:
df_2[1].unique()

array([None, 'バス(6分)都民農園下車徒歩1分', 'バス(19分)長久保下車徒歩2分', 'バス(7分)大泉風致地区下車徒歩5分',
       'バス(10分)関町郵便局前下車徒歩1分', 'バス(16分)新小71\u3000農業試験場下車徒歩3分',
       'バス(18分)土支田二丁目下車徒歩5分', 'バス(12分)比丘尼橋下車徒歩3分', 'バス(10分)団地入口下車徒歩3分',
       'バス(5分)善福寺下車徒歩1分', 'バス(15分)ふれあいの家下車', 'バス(13分)水元そよかぜ園下車徒歩1分',
       'バス(15分)水元4丁目下車徒歩28分', 'バス(25分)武蔵関駅下車徒歩7分', 'バス(15分)越後山下車徒歩4分',
       'バス(9分)西長久保下車徒歩1分', 'バス(16分)亀26\u3000松江下車徒歩5分',
       'バス(23分)葛飾車庫入口下車徒歩11分', 'バス(10分)大田文化の森下車徒歩3分',
       'バス(18分)幸田小学校下車徒歩3分', 'バス(11分)水元そよかぜ園下車徒歩6分',
       'バス(16分)佐野センター前下車徒歩34分', 'バス(15分)花畑団地下車徒歩10分',
       'バス(15分)水元そよかぜ園下車徒歩2分', 'バス(12分)六木団地下車徒歩3分',
       'バス(19分)大泉学園町八丁目下車徒歩2分', 'バス(18分)本木新道下車徒歩3分',
       'バス(13分)砧南中学校前下車徒歩4分', 'バス(10分)鹿骨五丁目下車徒歩3分', 'バス(3分)桑川町下車徒歩7分',
       'バス(12分)関町郵便局前下車徒歩5分', 'バス(5分)日産自動車前下車徒歩4分', 'バス(12分)水元2丁目下車徒歩2分',
       'バス(10分)富士公園下車徒歩3分', 'バス(18分)京成バス\u3000大杉3丁目下車徒歩1分',
       'バス(10分)都民農園下車徒歩3分', 'バス(12分)土支田二丁目下車徒歩6分', 'バス(12分)西長久保下車徒歩3分',
       'バス(14分)土支田三丁目下車徒歩3分', 'バス(9分)ハートアイランド南下車徒歩1分'

In [149]:
df_2_1 = df_2[1].str.split("分", expand=True)

In [150]:
df_2_1.head()

,0,1,2,3
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None


In [151]:
df_2_1[0].unique()

array([None, 'バス(6', 'バス(19', 'バス(7', 'バス(10', 'バス(16', 'バス(18', 'バス(12',
       'バス(5', 'バス(15', 'バス(13', 'バス(25', 'バス(9', 'バス(23', 'バス(11',
       'バス(3', 'バス(14', 'バス(21', 'バス(8', 'バス(20', 'バス(17', 'バス(1', 'バス(4',
       'バス(2', 'バス(27'], dtype=object)

In [152]:
total_data["アクセス_駅_1_バス_分"] = df_2_1[0].str.strip("バス(").astype(float)

In [153]:
total_data["アクセス_駅_1_バス_分"].unique()

array([nan,  6., 19.,  7., 10., 16., 18., 12.,  5., 15., 13., 25.,  9.,
       23., 11.,  3., 14., 21.,  8., 20., 17.,  1.,  4.,  2., 27.])

In [154]:
df_2_1[1] = df_2_1[1].str.strip(")")
df_2_1[1] = df_2_1[1].str.replace("\u3000", "")
df_2_1[1].unique()

array([None, '都民農園下車徒歩1', '長久保下車徒歩2', '大泉風致地区下車徒歩5', '関町郵便局前下車徒歩1',
       '新小71農業試験場下車徒歩3', '土支田二丁目下車徒歩5', '比丘尼橋下車徒歩3', '団地入口下車徒歩3',
       '善福寺下車徒歩1', 'ふれあいの家下車', '水元そよかぜ園下車徒歩1', '水元4丁目下車徒歩28', '武蔵関駅下車徒歩7',
       '越後山下車徒歩4', '西長久保下車徒歩1', '亀26松江下車徒歩5', '葛飾車庫入口下車徒歩11',
       '大田文化の森下車徒歩3', '幸田小学校下車徒歩3', '水元そよかぜ園下車徒歩6', '佐野センター前下車徒歩34',
       '花畑団地下車徒歩10', '水元そよかぜ園下車徒歩2', '六木団地下車徒歩3', '大泉学園町八丁目下車徒歩2',
       '本木新道下車徒歩3', '砧南中学校前下車徒歩4', '鹿骨五丁目下車徒歩3', '桑川町下車徒歩7',
       '関町郵便局前下車徒歩5', '日産自動車前下車徒歩4', '水元2丁目下車徒歩2', '富士公園下車徒歩3',
       '京成バス大杉3丁目下車徒歩1', '都民農園下車徒歩3', '土支田二丁目下車徒歩6', '西長久保下車徒歩3',
       '土支田三丁目下車徒歩3', 'ハートアイランド南下車徒歩1', '西水元2丁目下車徒歩2', '砧町下車徒歩3',
       '大泉風致地区下車徒歩3', '吉祥寺通り入口下車徒歩5', '水元神社前下車徒歩4', '水元3丁目下車徒歩1',
       '代沢一丁目下車徒歩5', '東小松川一丁目下車徒歩7', '鎌田下車徒歩4', '足立スポーツセンター下車徒歩2',
       '都民農園下車徒歩2', '扇2丁目下車徒歩3', '次大夫堀公園下車徒歩3', '関町3下車徒歩3', '大泉桜高校下車徒歩1',
       '水元四丁目下車徒歩3', '西水元二丁目下車徒歩38', '北大泉下車徒歩8', '新広橋下車徒歩1', '松本橋東詰下車徒歩3',
       '足立郷土博物館下車徒歩23', '大泉風致地区下車徒歩2', '関町北一丁目下車徒歩3', 

In [155]:
df_2_1_1 = df_2_1[1].str.split("下車徒歩", expand=True)

In [156]:
df_2_1_1.head()

,0,1
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None


In [157]:
df_2_1_1[0].unique()

array([None, '都民農園', '長久保', '大泉風致地区', '関町郵便局前', '新小71農業試験場', '土支田二丁目',
       '比丘尼橋', '団地入口', '善福寺', 'ふれあいの家下車', '水元そよかぜ園', '水元4丁目', '武蔵関駅',
       '越後山', '西長久保', '亀26松江', '葛飾車庫入口', '大田文化の森', '幸田小学校', '佐野センター前',
       '花畑団地', '六木団地', '大泉学園町八丁目', '本木新道', '砧南中学校前', '鹿骨五丁目', '桑川町',
       '日産自動車前', '水元2丁目', '富士公園', '京成バス大杉3丁目', '土支田三丁目', 'ハートアイランド南',
       '西水元2丁目', '砧町', '吉祥寺通り入口', '水元神社前', '水元3丁目', '代沢一丁目', '東小松川一丁目',
       '鎌田', '足立スポーツセンター', '扇2丁目', '次大夫堀公園', '関町3', '大泉桜高校', '水元四丁目',
       '西水元二丁目', '北大泉', '新広橋', '松本橋東詰', '足立郷土博物館', '関町北一丁目', '有27西水元一',
       '三原台北', '公園橋', '鹿骨', '大泉第三小', '宇奈根地区会館前', '綾21六木小', '六木都住',
       '鹿浜四丁目', '堀江団地', '鹿浜橋', '桜木橋北詰', '北出張所', '深沢小学校前徒歩1', '松本弁天', '川南',
       '入新井第四小学校', '関町南2丁目', '永安寺前', 'ふれあいの家', '天沼マーケット前', '元競馬場',
       '久保新田停留所', '樫の木公園前', '立野橋', '風致地区', 'スポーツランド入口', '大杉3丁目停', '永安寺',
       '西水元2丁目下車', '千歳台6丁目', 'ときわ橋', 'プロムナード荻窪', '大泉郵便局', 'あいぐみ公園前',
       '青梅街道営業所', '谷原中', '下宿', '谷原5丁目', '稲荷前', '西水元一丁目', '橋戸小学校',
       '上祖師谷四丁目', 

In [158]:
df_2_1_1[0] = df_2_1_1[0].str.strip("停歩5")

In [159]:
df_2_1_1[0].unique()

array([None, '都民農園', '長久保', '大泉風致地区', '関町郵便局前', '新小71農業試験場', '土支田二丁目',
       '比丘尼橋', '団地入口', '善福寺', 'ふれあいの家下車', '水元そよかぜ園', '水元4丁目', '武蔵関駅',
       '越後山', '西長久保', '亀26松江', '葛飾車庫入口', '大田文化の森', '幸田小学校', '佐野センター前',
       '花畑団地', '六木団地', '大泉学園町八丁目', '本木新道', '砧南中学校前', '鹿骨五丁目', '桑川町',
       '日産自動車前', '水元2丁目', '富士公園', '京成バス大杉3丁目', '土支田三丁目', 'ハートアイランド南',
       '西水元2丁目', '砧町', '吉祥寺通り入口', '水元神社前', '水元3丁目', '代沢一丁目', '東小松川一丁目',
       '鎌田', '足立スポーツセンター', '扇2丁目', '次大夫堀公園', '関町3', '大泉桜高校', '水元四丁目',
       '西水元二丁目', '北大泉', '新広橋', '松本橋東詰', '足立郷土博物館', '関町北一丁目', '有27西水元一',
       '三原台北', '公園橋', '鹿骨', '大泉第三小', '宇奈根地区会館前', '綾21六木小', '六木都住',
       '鹿浜四丁目', '堀江団地', '鹿浜橋', '桜木橋北詰', '北出張所', '深沢小学校前徒歩1', '松本弁天', '川南',
       '入新井第四小学校', '関町南2丁目', '永安寺前', 'ふれあいの家', '天沼マーケット前', '元競馬場',
       '久保新田停留所', '樫の木公園前', '立野橋', '風致地区', 'スポーツランド入口', '大杉3丁目', '永安寺',
       '西水元2丁目下車', '千歳台6丁目', 'ときわ橋', 'プロムナード荻窪', '大泉郵便局', 'あいぐみ公園前',
       '青梅街道営業所', '谷原中', '下宿', '谷原5丁目', '稲荷前', '西水元一丁目', '橋戸小学校',
       '上祖師谷四丁目', '

In [160]:
total_data.columns

Index(['所在地', 'アクセス', '間取り', '方角', '面積', '所在階', 'バス・トイレ', 'キッチン', '放送・通信',
       '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間', '築年数_月', '階建', '所在階_地下', '階建_地下',
       '納戸', '間取り_D', '間取り_K', '間取り_L', '間取り_R', '間取り_S', '間取り_数', '方角_南',
       '方角_東', '方角_西', '方角_北', '契約期間_定期借家', '契約期間_月', '所在地_区', 'アクセス_線_1',
       'アクセス_駅_1', 'アクセス_駅_1_徒歩_分', 'アクセス_駅_1_車_分', 'アクセス_駅_1_バス_分'],
      dtype='object')

In [161]:
total_data["アクセス_駅_1_バス停"] = df_2_1_1[0]

In [162]:
total_data["アクセス_駅_1_バス停_徒歩"] = df_2_1_1[1].astype(float)

### ignore df_2_1[2] is probably walking time after train

In [163]:
df_2_1[2] = df_2_1[2].replace(r'^\s*$', np.nan, regex=True)
df_2_1[2] = df_2_1[2].str.strip("下車徒歩").astype(float)
df_2_1[2].unique()

array([nan, 10., 35.])

In [164]:
total_data["アクセス"][df_2_1[2].notnull()]

9845     東急田園都市線\t桜新町駅\t/バス(1分)深沢小学校前　　徒歩1分下車徒歩10分
56577        西武池袋線\t大泉学園駅\t/バス(13分)長久保　停歩5分下車徒歩35分
Name: アクセス, dtype: object

In [165]:
total_data.columns

Index(['所在地', 'アクセス', '間取り', '方角', '面積', '所在階', 'バス・トイレ', 'キッチン', '放送・通信',
       '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間', '築年数_月', '階建', '所在階_地下', '階建_地下',
       '納戸', '間取り_D', '間取り_K', '間取り_L', '間取り_R', '間取り_S', '間取り_数', '方角_南',
       '方角_東', '方角_西', '方角_北', '契約期間_定期借家', '契約期間_月', '所在地_区', 'アクセス_線_1',
       'アクセス_駅_1', 'アクセス_駅_1_徒歩_分', 'アクセス_駅_1_車_分', 'アクセス_駅_1_バス_分',
       'アクセス_駅_1_バス停', 'アクセス_駅_1_バス停_徒歩'],
      dtype='object')

In [166]:
total_data["アクセス_駅_1_徒歩_分"] = total_data["アクセス_駅_1_徒歩_分"].combine_first(df_2_1[2])

In [167]:
df_2_1[3].unique()

array([None, ''], dtype=object)

## df[3]

In [168]:
df[3].unique()

array(['', None], dtype=object)

## df[4]

In [169]:
df[4].unique()

array(['埼京線', '有楽町線', '京王線', '丸ノ内線(池袋-荻窪)', '常磐線', '都電荒川線', '西武池袋線',
       '東急東横線', '都営三田線', '都営浅草線', '京成本線', '千代田線', '都営大江戸線', '副都心線',
       '都営新宿線', '日比谷線', '西武新宿線', '京葉線(東京-蘇我)', '京急本線', '小田急小田原線', '山手線',
       '東西線', '東急田園都市線', '京成金町線', '南北線', '東急池上線', '常磐線快速', '東武スカイツリーライン',
       '銀座線', 'つくばエクスプレス', '東急大井町線', '総武本線', None, '丸ノ内線(中野坂上-方南町)',
       '東急世田谷線', '総武線・中央線(各停)', '京王新線', '京成押上線', '京王井の頭線', '京浜東北線',
       '東急目黒線', 'ゆりかもめ', '東武東上線', 'りんかい線', '日暮里・舎人ライナー', '東武伊勢崎線(押上-曳舟)',
       '中央線(快速)', '東急多摩川線', '半蔵門線', '東武亀戸線', '北総線', '東武大師線', '京急空港線',
       '西武有楽町線', '高崎線', '東京モノレール羽田線', '西武豊島線', '東北本線(宇都宮線)', '東海道本線',
       '横須賀線', '埼玉高速鉄道線', '湘南新宿ライン(東北本線-横須賀線)', '京急大師線', '三田線', '京成成田空港線',
       'ディズニーリゾートライン', '東海道新幹線', '小76/松本橋東詰\u30006分', '南武線(川崎-立川)'],
      dtype=object)

In [170]:
df[4][df[4].str.contains("線") == False].unique()

array(['東武スカイツリーライン', 'つくばエクスプレス', 'ゆりかもめ', '日暮里・舎人ライナー', 'ディズニーリゾートライン',
       '小76/松本橋東詰\u30006分'], dtype=object)

In [171]:
df[4] = df[4].replace("小76/松本橋東詰\u30006分", None, regex=False)

In [172]:
df[4][df[4].str.contains("線") == False].unique()

array(['東武スカイツリーライン', 'つくばエクスプレス', 'ゆりかもめ', '日暮里・舎人ライナー', 'ディズニーリゾートライン'],
      dtype=object)

In [173]:
total_data.columns

Index(['所在地', 'アクセス', '間取り', '方角', '面積', '所在階', 'バス・トイレ', 'キッチン', '放送・通信',
       '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間', '築年数_月', '階建', '所在階_地下', '階建_地下',
       '納戸', '間取り_D', '間取り_K', '間取り_L', '間取り_R', '間取り_S', '間取り_数', '方角_南',
       '方角_東', '方角_西', '方角_北', '契約期間_定期借家', '契約期間_月', '所在地_区', 'アクセス_線_1',
       'アクセス_駅_1', 'アクセス_駅_1_徒歩_分', 'アクセス_駅_1_車_分', 'アクセス_駅_1_バス_分',
       'アクセス_駅_1_バス停', 'アクセス_駅_1_バス停_徒歩'],
      dtype='object')

In [174]:
total_data["アクセス_線_2"] = df[4]

## df[5]

In [175]:
df[5].unique()

array(['板橋駅', '月島駅', '代田橋駅', '新高円寺駅', '金町(東京都)駅', '三ノ輪橋駅', '保谷駅', '都立大学駅',
       '白山(東京都)駅', '新板橋駅', '馬込駅', '江戸川駅', '表参道駅', '南阿佐ケ谷駅', '両国(都営線)駅',
       '笹塚駅', '氷川台駅', '船堀駅', '四ツ谷駅', '茅場町駅', '野方駅', '湯島駅', '越中島駅', '平和島駅',
       '京成小岩駅', '東長崎駅', '仲御徒町駅', '北綾瀬駅', '一之江駅', '東北沢駅', '目白駅', '西葛西駅',
       '用賀駅', '国立競技場駅', 'お花茶屋駅', '石神井公園駅', '荒川一中前駅', '落合南長崎駅', '京成金町駅',
       '後楽園駅', '浅草駅', '西馬込駅', '千鳥町駅', '大泉学園駅', '南千住駅', '曳舟駅', '渋谷駅',
       '新江古田駅', '西新宿五丁目駅', '千歳烏山駅', '浅草(TX)駅', '駒沢大学駅', '緑が丘(東京都)駅',
       '錦糸町駅', '八丁堀(東京都)駅', '馬喰町駅', '本駒込駅', '大森町駅', '祖師ケ谷大蔵駅', None,
       '蓮沼駅', '青井駅', '石川台駅', '中野新橋駅', '町屋(東京メトロ)駅', '栄町(東京都)駅', '巣鴨新田駅',
       '瑞江駅', '西大島駅', '春日(東京都)駅', '梅島駅', '千駄木駅', '北赤羽駅', '東高円寺駅', '上町駅',
       '西ケ原駅', '地下鉄成増駅', '若林(東京都)駅', '広尾駅', '門前仲町駅', '池袋駅', '押上駅', '熊野前駅',
       '荻窪駅', '王子駅前駅', '青物横丁駅', '方南町駅', '京成曳舟駅', '水道橋駅', '大島(東京都)駅',
       '高井戸駅', '日暮里駅', '六町駅', '乃木坂駅', '青砥駅', '西巣鴨駅', '汐留駅', '要町駅', '上野毛駅',
       '板橋区役所前駅', '中野(東京都)駅', '西小山駅', '麻布十番駅', '王子神谷駅', '狛江駅',

In [176]:
df[5][df[5].str.contains("駅") == False].unique()

array([], dtype=object)

In [177]:
total_data["アクセス_駅_2"] = df[5]

## df[6]

In [178]:
df[6].unique()

array(['徒歩14分', '徒歩9分', '徒歩7分', '徒歩3分', '徒歩10分', '徒歩25分', '徒歩16分', '徒歩4分',
       '徒歩11分', '徒歩12分', '徒歩21分', '徒歩30分', '徒歩6分', '徒歩15分', '徒歩23分',
       '徒歩8分', '徒歩18分', '徒歩2分', '徒歩20分', '徒歩19分', '徒歩13分', '徒歩5分', None,
       '徒歩28分', '徒歩24分', '徒歩17分', '/バス(20分)都民農園下車徒歩1分',
       '/バス(17分)長久保下車徒歩2分', '/バス(10分)東葛西八丁目下車徒歩34分',
       '/バス(10分)最寄りのバス停下車徒歩14分', '徒歩22分', '/バス(7分)三軒寺下車徒歩1分',
       '車5.2km(18分)', '徒歩31分', '徒歩26分', '徒歩27分', '/バス(5分)大泉北中学校入口下車徒歩7分',
       '/バス(15分)足立車検場下車徒歩2分', '徒歩1分', '徒歩42分', '/バス(15分)東小岩一丁目下車徒歩2分',
       '/バス(27分)石神井郵便局停留所下車徒歩7分', '/バス(15分)谷中中入口下車徒歩3分', '徒歩70分', '徒歩36分',
       '/バス(16分)土支田二丁目下車徒歩5分', '/バス(7分)上祖師谷4丁目下車徒歩2分', '徒歩37分',
       '/バス(10分)本州製紙裏門下車徒歩2分', '/バス(19分)新小岩1丁目下車徒歩8分', '徒歩33分',
       '/バス(18分)比丘尼橋下車徒歩3分', '/バス(5分)日産自動車前下車徒歩4分', '/バス(10分)中央図書館下車徒歩3分',
       '/バス(15分)大谷田下車徒歩2分', '徒歩29分', '/バス(4分)善福寺下車徒歩1分',
       '/バス(12分)谷原二丁目下車徒歩16分', '/バス(10分)鹿本橋下車徒歩6分', '徒歩32分',
       '/バス(13分)水元そよかぜ園下車徒歩1分', '/バス(15分)鹿骨五丁目下車徒歩7分',
       '/バス(21分)

In [179]:
df_6 = df[6].str.split("/", expand=True)
df_6.head(3)

,0,1
0,徒歩14分,None
1,徒歩9分,None
2,徒歩7分,None


In [180]:
df_6[0].unique()

array(['徒歩14分', '徒歩9分', '徒歩7分', '徒歩3分', '徒歩10分', '徒歩25分', '徒歩16分', '徒歩4分',
       '徒歩11分', '徒歩12分', '徒歩21分', '徒歩30分', '徒歩6分', '徒歩15分', '徒歩23分',
       '徒歩8分', '徒歩18分', '徒歩2分', '徒歩20分', '徒歩19分', '徒歩13分', '徒歩5分', None,
       '徒歩28分', '徒歩24分', '徒歩17分', '', '徒歩22分', '車5.2km(18分)', '徒歩31分',
       '徒歩26分', '徒歩27分', '徒歩1分', '徒歩42分', '徒歩70分', '徒歩36分', '徒歩37分',
       '徒歩33分', '徒歩29分', '徒歩32分', '徒歩38分', '徒歩35分', '車3.1km(9分)', '徒歩55分',
       '徒歩41分', '徒歩45分', '徒歩34分', '徒歩44分', '車4.6km(17分)', '車3.1km(14分)',
       '車2.48km(7分)', '徒歩74分', '車3.6km(15分)', '徒歩43分', '徒歩39分', '徒歩48分',
       '徒歩65分', '車2.8km(5分)', '徒歩63分', '車5.9km(17分)', '徒歩156分', '徒歩47分',
       '徒歩40分', '車4.3km(12分)', '徒歩78分', '車2.78km(7分)', '徒歩75分',
       '車3.5km(10分)', '徒歩52分', '車3.2km(9分)', '車7.3km(25分)', '徒歩62分',
       '車2.8km(10分)', '車3.9km(13分)', '車2.58km(7分)', '徒歩58分', '徒歩113分',
       '車6.4km(22分)', '車3.3km(12分)', '車3.6km(11分)', '徒歩69分', '徒歩46分',
       '車4.1km(8分)', '徒歩68分', '徒歩51分', '車5.4km(18分)', '徒歩60分',
       '車3km

In [181]:
total_data["アクセス_駅_2_徒歩_分"] = df_6[0].where(df_6[0].str.contains("徒歩", regex=False))
total_data["アクセス_駅_2_徒歩_分"].unique()

array(['徒歩14分', '徒歩9分', '徒歩7分', '徒歩3分', '徒歩10分', '徒歩25分', '徒歩16分', '徒歩4分',
       '徒歩11分', '徒歩12分', '徒歩21分', '徒歩30分', '徒歩6分', '徒歩15分', '徒歩23分',
       '徒歩8分', '徒歩18分', '徒歩2分', '徒歩20分', '徒歩19分', '徒歩13分', '徒歩5分', nan,
       '徒歩28分', '徒歩24分', '徒歩17分', '徒歩22分', '徒歩31分', '徒歩26分', '徒歩27分',
       '徒歩1分', '徒歩42分', '徒歩70分', '徒歩36分', '徒歩37分', '徒歩33分', '徒歩29分',
       '徒歩32分', '徒歩38分', '徒歩35分', '徒歩55分', '徒歩41分', '徒歩45分', '徒歩34分',
       '徒歩44分', '徒歩74分', '徒歩43分', '徒歩39分', '徒歩48分', '徒歩65分', '徒歩63分',
       '徒歩156分', '徒歩47分', '徒歩40分', '徒歩78分', '徒歩75分', '徒歩52分', '徒歩62分',
       '徒歩58分', '徒歩113分', '徒歩69分', '徒歩46分', '徒歩68分', '徒歩51分', '徒歩60分',
       '徒歩157分', '徒歩86分', '徒歩53分', '徒歩54分', '徒歩73分', '徒歩56分'],
      dtype=object)

In [182]:
total_data["アクセス_駅_2_徒歩_分"] = total_data["アクセス_駅_2_徒歩_分"].str.strip("徒歩分").astype(float)

In [183]:
total_data["アクセス_駅_2_徒歩_分"].unique()

array([ 14.,   9.,   7.,   3.,  10.,  25.,  16.,   4.,  11.,  12.,  21.,
        30.,   6.,  15.,  23.,   8.,  18.,   2.,  20.,  19.,  13.,   5.,
        nan,  28.,  24.,  17.,  22.,  31.,  26.,  27.,   1.,  42.,  70.,
        36.,  37.,  33.,  29.,  32.,  38.,  35.,  55.,  41.,  45.,  34.,
        44.,  74.,  43.,  39.,  48.,  65.,  63., 156.,  47.,  40.,  78.,
        75.,  52.,  62.,  58., 113.,  69.,  46.,  68.,  51.,  60., 157.,
        86.,  53.,  54.,  73.,  56.])

In [184]:
total_data["アクセス_駅_2_車_分"] = df_6[0].where(df_6[0].str.contains("徒歩", regex=False)==False)

In [185]:
total_data["アクセス_駅_2_車_分"] = total_data["アクセス_駅_2_車_分"].replace(r'^\s*$', np.nan, regex=True)

In [186]:
total_data["アクセス_駅_2_車_分"].unique()

array([nan, '車5.2km(18分)', '車3.1km(9分)', '車4.6km(17分)', '車3.1km(14分)',
       '車2.48km(7分)', '車3.6km(15分)', '車2.8km(5分)', '車5.9km(17分)',
       '車4.3km(12分)', '車2.78km(7分)', '車3.5km(10分)', '車3.2km(9分)',
       '車7.3km(25分)', '車2.8km(10分)', '車3.9km(13分)', '車2.58km(7分)',
       '車6.4km(22分)', '車3.3km(12分)', '車3.6km(11分)', '車4.1km(8分)',
       '車5.4km(18分)', '車3km(12分)', '車5.5km(20分)', '車4.3km(14分)',
       '車7.4km(21分)', '車2.9km(9分)', '車3km(10分)', '車5.8km(20分)'],
      dtype=object)

In [187]:
total_data["アクセス_駅_2_車_分"] = total_data["アクセス_駅_2_車_分"].str.split("(", expand=True)[1].str.strip("分)").astype(float)

In [188]:
total_data["アクセス_駅_2_車_分"].unique()

array([nan, 18.,  9., 17., 14.,  7., 15.,  5., 12., 10., 25., 13., 22.,
       11.,  8., 20., 21.])

In [189]:
df_6[1].unique()

array([None, 'バス(20分)都民農園下車徒歩1分', 'バス(17分)長久保下車徒歩2分',
       'バス(10分)東葛西八丁目下車徒歩34分', 'バス(10分)最寄りのバス停下車徒歩14分', 'バス(7分)三軒寺下車徒歩1分',
       'バス(5分)大泉北中学校入口下車徒歩7分', 'バス(15分)足立車検場下車徒歩2分',
       'バス(15分)東小岩一丁目下車徒歩2分', 'バス(27分)石神井郵便局停留所下車徒歩7分',
       'バス(15分)谷中中入口下車徒歩3分', 'バス(16分)土支田二丁目下車徒歩5分', 'バス(7分)上祖師谷4丁目下車徒歩2分',
       'バス(10分)本州製紙裏門下車徒歩2分', 'バス(19分)新小岩1丁目下車徒歩8分', 'バス(18分)比丘尼橋下車徒歩3分',
       'バス(5分)日産自動車前下車徒歩4分', 'バス(10分)中央図書館下車徒歩3分', 'バス(15分)大谷田下車徒歩2分',
       'バス(4分)善福寺下車徒歩1分', 'バス(12分)谷原二丁目下車徒歩16分', 'バス(10分)鹿本橋下車徒歩6分',
       'バス(13分)水元そよかぜ園下車徒歩1分', 'バス(15分)鹿骨五丁目下車徒歩7分', 'バス(21分)滋雲堂前下車徒歩1分',
       'バス(7分)高円寺駅北口下車徒歩6分', 'バス(12分)新小51\u3000東上平井下車徒歩1分',
       'バス(15分)東武バス\u3000西新井駅西口下車徒歩5分', 'バス(20分)越後山下車徒歩4分',
       'バス(27分)第五上石神井住宅下車徒歩6分', 'バス(20分)辰沼団地入口下車徒歩3分',
       'バス(10分)区立八中下車徒歩2分', 'バス(13分)新蒲田三丁目下車徒歩3分', 'バス(15分)佐野センター前下車徒歩2分',
       'バス(25分)関町北町学校下車徒歩8分', 'バス(15分)都営住宅下車徒歩1分', 'バス(21分)新島橋下車徒歩3分',
       'バス(27分)西長久保(バス)下車徒歩1分', 'バス(13分)瑞江斎場下車徒歩20分',
       'バス(28分)西武車庫前下車徒歩

In [190]:
df_6_1 = df_6[1].str.split("分", expand=True)

In [191]:
df_6_1.head()

,0,1,2
0,None,None,None
1,None,None,None
2,None,None,None
3,None,None,None
4,None,None,None


In [192]:
df_6_1[0].unique()

array([None, 'バス(20', 'バス(17', 'バス(10', 'バス(7', 'バス(5', 'バス(15', 'バス(27',
       'バス(16', 'バス(19', 'バス(18', 'バス(4', 'バス(12', 'バス(13', 'バス(21',
       'バス(25', 'バス(28', 'バス(8', 'バス(22', 'バス(26', 'バス(35', 'バス(31',
       'バス(11', 'バス(14', 'バス(6', 'バス(2', 'バス(9', 'バス(29', 'バス(24',
       'バス(30', 'バス(36', 'バス(60', 'バス(38', 'バス(33', 'バス(40', 'バス(41',
       'バス(3', 'バス(52', 'バス(1', 'バス(23', 'バス(55', 'バス(32', 'バス(37'],
      dtype=object)

In [193]:
total_data["アクセス_駅_2_バス_分"] = df_6_1[0].str.strip("バス(").astype(float)

In [194]:
total_data["アクセス_駅_2_バス_分"].unique()

array([nan, 20., 17., 10.,  7.,  5., 15., 27., 16., 19., 18.,  4., 12.,
       13., 21., 25., 28.,  8., 22., 26., 35., 31., 11., 14.,  6.,  2.,
        9., 29., 24., 30., 36., 60., 38., 33., 40., 41.,  3., 52.,  1.,
       23., 55., 32., 37.])

In [195]:
df_6_1[1] = df_6_1[1].str.strip(")")
df_6_1[1] = df_6_1[1].str.replace("\u3000", "")
df_6_1[1].unique()

array([None, '都民農園下車徒歩1', '長久保下車徒歩2', '東葛西八丁目下車徒歩34', '最寄りのバス停下車徒歩14',
       '三軒寺下車徒歩1', '大泉北中学校入口下車徒歩7', '足立車検場下車徒歩2', '東小岩一丁目下車徒歩2',
       '石神井郵便局停留所下車徒歩7', '谷中中入口下車徒歩3', '土支田二丁目下車徒歩5', '上祖師谷4丁目下車徒歩2',
       '本州製紙裏門下車徒歩2', '新小岩1丁目下車徒歩8', '比丘尼橋下車徒歩3', '日産自動車前下車徒歩4',
       '中央図書館下車徒歩3', '大谷田下車徒歩2', '善福寺下車徒歩1', '谷原二丁目下車徒歩16', '鹿本橋下車徒歩6',
       '水元そよかぜ園下車徒歩1', '鹿骨五丁目下車徒歩7', '滋雲堂前下車徒歩1', '高円寺駅北口下車徒歩6',
       '新小51東上平井下車徒歩1', '東武バス西新井駅西口下車徒歩5', '越後山下車徒歩4', '第五上石神井住宅下車徒歩6',
       '辰沼団地入口下車徒歩3', '区立八中下車徒歩2', '新蒲田三丁目下車徒歩3', '佐野センター前下車徒歩2',
       '関町北町学校下車徒歩8', '都営住宅下車徒歩1', '新島橋下車徒歩3', '西長久保(バス)下車徒歩1',
       '瑞江斎場下車徒歩20', '西武車庫前下車徒歩1', '北出張所下車徒歩2', '永安寺下車徒歩6', '武蔵関駅前下車徒歩4',
       '葛西駅下車徒歩21', '南葛西第二小学校前下車徒歩1', '幸田小学校下車徒歩3', '関原2丁目下車徒歩3',
       '小関下車徒歩9', '小関下車徒歩4', '入谷新道(バス)下車徒歩15', '東映撮影所前(バス)下車徒歩14',
       '樫の木公園前下車徒歩6', '水元そよかぜ園下車徒歩2', '長久保(東京都)下車徒歩2', 'ハートアイランド南下車徒歩2',
       '関町北1丁目下車徒歩3', '小関下車徒歩5', '花畑3町目下車徒歩6', '岡本3丁目下車徒歩6', '江北一丁目下車徒歩8',
       '関町北小学校下車徒歩8', '永安寺前下車徒歩6', 

In [196]:
df_6_1_1 = df_6_1[1].str.split("下車徒歩", expand=True)

In [197]:
df_6_1_1.head()

,0,1
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None


In [198]:
df_6_1_1[0].unique()

array([None, '都民農園', '長久保', '東葛西八丁目', '最寄りのバス停', '三軒寺', '大泉北中学校入口',
       '足立車検場', '東小岩一丁目', '石神井郵便局停留所', '谷中中入口', '土支田二丁目', '上祖師谷4丁目',
       '本州製紙裏門', '新小岩1丁目', '比丘尼橋', '日産自動車前', '中央図書館', '大谷田', '善福寺',
       '谷原二丁目', '鹿本橋', '水元そよかぜ園', '鹿骨五丁目', '滋雲堂前', '高円寺駅北口', '新小51東上平井',
       '東武バス西新井駅西口', '越後山', '第五上石神井住宅', '辰沼団地入口', '区立八中', '新蒲田三丁目',
       '佐野センター前', '関町北町学校', '都営住宅', '新島橋', '西長久保(バス)', '瑞江斎場', '西武車庫前',
       '北出張所', '永安寺', '武蔵関駅前', '葛西駅', '南葛西第二小学校前', '幸田小学校', '関原2丁目', '小関',
       '入谷新道(バス)', '東映撮影所前(バス)', '樫の木公園前', '長久保(東京都)', 'ハートアイランド南',
       '関町北1丁目', '花畑3町目', '岡本3丁目', '江北一丁目', '関町北小学校', '永安寺前', '下井草二丁目',
       '国立西が丘競技場北門', '青梅街道営業所', '松の木', '水元2丁目', '千住車庫前', '水元中',
       '京成バス大杉3丁目', '次大夫堀公園', '天沼マーケット前', '西水元2丁目', '品川シーサイド駅前', '野毛二丁目',
       '半田稲荷前', '足立総合スポーツセンター前', '北糀谷', '水元3丁目', '西新井駅西口', '関町北一丁目',
       '住宅前', '本郷花街道', '加平2丁目', '保木間一丁目', '六木小学校', '東小松川一丁目',
       '上石神井イ中学校入口', '水元特別支援学校前', '鎌田', '恵32五本木', '中図師', '菅原橋', '中郷',
       'NTT技研前', '小松川警察署前', 

In [199]:
df_6_1_1[0] = df_6_1_1[0].str.strip("下車")
# df_6_1_1[0] = df_6_1_1[0].str.strip("駅")

In [200]:
df_6_1_1[0].unique()

array([None, '都民農園', '長久保', '東葛西八丁目', '最寄りのバス停', '三軒寺', '大泉北中学校入口',
       '足立車検場', '東小岩一丁目', '石神井郵便局停留所', '谷中中入口', '土支田二丁目', '上祖師谷4丁目',
       '本州製紙裏門', '新小岩1丁目', '比丘尼橋', '日産自動車前', '中央図書館', '大谷田', '善福寺',
       '谷原二丁目', '鹿本橋', '水元そよかぜ園', '鹿骨五丁目', '滋雲堂前', '高円寺駅北口', '新小51東上平井',
       '東武バス西新井駅西口', '越後山', '第五上石神井住宅', '辰沼団地入口', '区立八中', '新蒲田三丁目',
       '佐野センター前', '関町北町学校', '都営住宅', '新島橋', '西長久保(バス)', '瑞江斎場', '西武車庫前',
       '北出張所', '永安寺', '武蔵関駅前', '葛西駅', '南葛西第二小学校前', '幸田小学校', '関原2丁目', '小関',
       '入谷新道(バス)', '東映撮影所前(バス)', '樫の木公園前', '長久保(東京都)', 'ハートアイランド南',
       '関町北1丁目', '花畑3町目', '岡本3丁目', '江北一丁目', '関町北小学校', '永安寺前', '井草二丁目',
       '国立西が丘競技場北門', '青梅街道営業所', '松の木', '水元2丁目', '千住車庫前', '水元中',
       '京成バス大杉3丁目', '次大夫堀公園', '天沼マーケット前', '西水元2丁目', '品川シーサイド駅前', '野毛二丁目',
       '半田稲荷前', '足立総合スポーツセンター前', '北糀谷', '水元3丁目', '西新井駅西口', '関町北一丁目',
       '住宅前', '本郷花街道', '加平2丁目', '保木間一丁目', '六木小学校', '東小松川一丁目',
       '上石神井イ中学校入口', '水元特別支援学校前', '鎌田', '恵32五本木', '中図師', '菅原橋', '中郷',
       'NTT技研前', '小松川警察署前', '

In [201]:
for i in df_6_1_1[0].unique():
    print(i)

None
都民農園
長久保
東葛西八丁目
最寄りのバス停
三軒寺
大泉北中学校入口
足立車検場
東小岩一丁目
石神井郵便局停留所
谷中中入口
土支田二丁目
上祖師谷4丁目
本州製紙裏門
新小岩1丁目
比丘尼橋
日産自動車前
中央図書館
大谷田
善福寺
谷原二丁目
鹿本橋
水元そよかぜ園
鹿骨五丁目
滋雲堂前
高円寺駅北口
新小51東上平井
東武バス西新井駅西口
越後山
第五上石神井住宅
辰沼団地入口
区立八中
新蒲田三丁目
佐野センター前
関町北町学校
都営住宅
新島橋
西長久保(バス)
瑞江斎場
西武車庫前
北出張所
永安寺
武蔵関駅前
葛西駅
南葛西第二小学校前
幸田小学校
関原2丁目
小関
入谷新道(バス)
東映撮影所前(バス)
樫の木公園前
長久保(東京都)
ハートアイランド南
関町北1丁目
花畑3町目
岡本3丁目
江北一丁目
関町北小学校
永安寺前
井草二丁目
国立西が丘競技場北門
青梅街道営業所
松の木
水元2丁目
千住車庫前
水元中
京成バス大杉3丁目
次大夫堀公園
天沼マーケット前
西水元2丁目
品川シーサイド駅前
野毛二丁目
半田稲荷前
足立総合スポーツセンター前
北糀谷
水元3丁目
西新井駅西口
関町北一丁目
住宅前
本郷花街道
加平2丁目
保木間一丁目
六木小学校
東小松川一丁目
上石神井イ中学校入口
水元特別支援学校前
鎌田
恵32五本木
中図師
菅原橋
中郷
NTT技研前
小松川警察署前
駅
久保新田
水元四丁目
飯塚橋
新広橋
扇2丁目
東武バス四ッ家町
仲町西組
都バス弥生町
本木2丁目
足立郷土博物館
名主屋敷
関町北四丁目
野方三丁目
有27西水元二
谷中3丁目
千歳台4丁目
上石神井駅前
船堀駅前
保木間町
光が丘六丁目
砧本村
ハートアイランド西(バス)
新田3丁目
谷中三丁目
五丁目住宅
大泉風致地区
佐野二丁目
六木小学校前
六ツ木団地
保谷駅南口
幸町:国際興業バス
大森東5丁目
新小岩駅前
江戸川区役所前
大杉第二小学校
代沢1丁目
石神井団地入口
公園橋
大泉学園駅南口
二丁目西
都バスドナウ通り
高松五丁目
西村
鹿骨3丁目
花畑団地停
いこいの家入口
大井第一小学校
西長久保
目黒五丁目
代沢一丁目
神明町
南葛西中学校前
舎人二ツ橋
学芸大学駅
都営バスドナウ通り
篠崎駅
菊池外科胃腸科
武蔵関駅
水元4丁目
東

In [202]:
total_data.columns

Index(['所在地', 'アクセス', '間取り', '方角', '面積', '所在階', 'バス・トイレ', 'キッチン', '放送・通信',
       '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間', '築年数_月', '階建', '所在階_地下', '階建_地下',
       '納戸', '間取り_D', '間取り_K', '間取り_L', '間取り_R', '間取り_S', '間取り_数', '方角_南',
       '方角_東', '方角_西', '方角_北', '契約期間_定期借家', '契約期間_月', '所在地_区', 'アクセス_線_1',
       'アクセス_駅_1', 'アクセス_駅_1_徒歩_分', 'アクセス_駅_1_車_分', 'アクセス_駅_1_バス_分',
       'アクセス_駅_1_バス停', 'アクセス_駅_1_バス停_徒歩', 'アクセス_線_2', 'アクセス_駅_2',
       'アクセス_駅_2_徒歩_分', 'アクセス_駅_2_車_分', 'アクセス_駅_2_バス_分'],
      dtype='object')

In [203]:
total_data["アクセス_駅_2_バス停"] = df_6_1_1[0]

In [204]:
total_data["アクセス_駅_2_バス停_徒歩"] = df_6_1_1[1].astype(float)

In [205]:
total_data.iloc[:, 32:][total_data["アクセス_駅_1_車_分"].notnull()]

,アクセス_線_1,アクセス_駅_1,アクセス_駅_1_徒歩_分,アクセス_駅_1_車_分,アクセス_駅_1_バス_分,アクセス_駅_1_バス停,アクセス_駅_1_バス停_徒歩,アクセス_線_2,アクセス_駅_2,アクセス_駅_2_徒歩_分,アクセス_駅_2_車_分,アクセス_駅_2_バス_分,アクセス_駅_2_バス停,アクセス_駅_2_バス停_徒歩
12581,山手線,日暮里駅,NaN,15.0,NaN,None,NaN,千代田線,町屋(東京メトロ)駅,8.0,NaN,NaN,None,NaN
34224,常磐線,金町(東京都)駅,NaN,10.0,NaN,None,NaN,常磐線,亀有駅,NaN,NaN,24.0,水元総合スポーツセンター,6.0
37603,千代田線,北綾瀬駅,NaN,11.0,NaN,None,NaN,常磐線,亀有駅,NaN,NaN,19.0,西水元二丁目,3.0


In [206]:
train["アクセス"][12581]

'山手線\t日暮里駅\t車3km(15分)\t\t千代田線\t町屋(東京メトロ)駅\t徒歩8分\t\t都電荒川線\t町屋駅前駅\t徒歩7分'

# df[7]

In [207]:
df[7].unique()

array(['', None], dtype=object)

## df[8]

In [220]:
df[8].unique().shape

(470,)

In [112]:
# df[8] = df[8].str.replace("\u3000", "")

In [221]:
df[8].str.split("/", expand=True)[0].unique().shape

(426,)

In [222]:
df[8].str.split("/", expand=True)[1].unique().shape

(77,)

### 2 distinct formats

In [243]:
df[8][df[8].str.contains("分") == True].shape

(557,)

In [244]:
df[8][df[8].str.contains("分") == False].shape

(55692,)

In [229]:
df[8][df[8].isnull()].shape

(6483,)

In [245]:
557+55692+6483 == total_data.shape[0]

True

In [248]:
total_data["アクセス_線_3"] = df[8].where(df[8].str.contains("分") == False)

In [320]:
station_names_without_sen = ['東武スカイツリーライン', '日暮里・舎人ライナー', 'ゆりかもめ', 'つくばエクスプレス']

In [504]:
temp_df = total_data["アクセス"]

temp_df = temp_df.str.replace("\u3000\u3000", "\u3000", regex=False)

for i in range(10):
    temp_df = temp_df.str.replace(str(i) + "\u3000", str(i), regex=False)
    
temp_df = temp_df.str.replace(")\u3000", "", regex=False)

temp_df = temp_df.str.replace("(東京都)", "", regex=False)
temp_df = temp_df.str.replace("熊野前(舎人ライナー)駅", "熊野前舎人ライナ駅", regex=False)
temp_df = temp_df.str.replace("町屋(東京メトロ)駅", "町屋東京メトロ駅", regex=False)
temp_df = temp_df.str.replace("町屋(東京メトロ)", "町屋東京メトロ駅", regex=False)
temp_df = temp_df.str.replace("豊島園(西武線)駅", "豊島園西武駅", regex=False)
temp_df = temp_df.str.replace("早稲田(東京メトロ)駅", "早稲田東京メトロ駅", regex=False)
temp_df = temp_df.str.replace("浅草(TX)駅", "浅草TX駅", regex=False)
temp_df = temp_df.str.replace("豊島園(都営線)駅", "豊島園都営駅", regex=False)
temp_df = temp_df.str.replace("浦安(千葉県)駅", "浦安千葉県駅", regex=False)
temp_df = temp_df.str.replace("早稲田(都電荒川線)駅", "早稲田都電荒川駅", regex=False)
temp_df = temp_df.str.replace("西日暮里(舎人ライナー)駅", "西日暮里舎人ライナ駅", regex=False)
temp_df = temp_df.str.replace("町屋(京成線)駅", "町屋京成駅", regex=False)
temp_df = temp_df.str.replace("両国(都営線)駅", "両国都営駅", regex=False)
temp_df = temp_df.str.replace("町屋駅前駅", "町屋駅", regex=False)
temp_df = temp_df.str.replace("大塚駅前駅", "大塚駅", regex=False)
temp_df = temp_df.str.replace("王子駅前駅", "王子駅", regex=False)
temp_df = temp_df.str.replace("日暮里(舎人ライナー)駅", "日暮里舎人ライナ駅", regex=False)

temp_df = temp_df.str.replace("総武線・中央線(各停)", "総武中央各停線", regex=False)
temp_df = temp_df.str.replace("総武線・中央線(快速)", "総武中央快速線", regex=False)
temp_df = temp_df.str.replace("中央線(各停)", "中央各停線", regex=False)
temp_df = temp_df.str.replace("中央線(快速)", "中央快速線", regex=False)
temp_df = temp_df.str.replace("東武スカイツリーライン", "東武スカイツリーライン線", regex=False)
temp_df = temp_df.str.replace("日暮里・舎人ライナー", "日暮里・舎人ライナー線", regex=False)
temp_df = temp_df.str.replace("ゆりかもめ", "ゆりかもめ線", regex=False)
temp_df = temp_df.str.replace("つくばエクスプレス", "つくばエクスプレス線", regex=False)
temp_df = temp_df.str.replace("丸ノ内線(中野坂上-方南町)", "丸ノ内中野坂上方南町線", regex=False)
temp_df = temp_df.str.replace("丸ノ内線(池袋-荻窪)", "丸ノ内池袋荻窪線", regex=False)
temp_df = temp_df.str.replace("常磐線快速", "常磐快速線", regex=False)
temp_df = temp_df.str.replace("京葉線(東京-蘇我)", "京葉東京蘇我線", regex=False)
temp_df = temp_df.str.replace("東武伊勢崎線(押上-曳舟)", "東武伊勢崎押上曳舟線", regex=False)
temp_df = temp_df.str.replace("東北本線(宇都宮線)", "東北本宇都宮線", regex=False)
temp_df = temp_df.str.replace("湘南新宿ライン(東北本線-横須賀線)", "湘南新宿ライン東北本横須賀線", regex=False)
temp_df = temp_df.str.replace("湘南新宿ライン(高崎線-東海道本線)", "湘南新宿ライン高崎東海道本線", regex=False)
temp_df = temp_df.str.replace("中央本線(東京-松本)", "中央本東京松本線", regex=False)
temp_df = temp_df.str.replace("南武線(川崎-立川)", "南武川崎立川線", regex=False)


temp_df = temp_df.str.replace("線", "線\t", regex=False)
temp_df = temp_df.str.replace("駅", "駅\t", regex=False)
temp_df = temp_df.str.replace("徒歩", "\t徒歩", regex=False)
temp_df = temp_df.str.replace("徒歩", "\t徒歩", regex=False)
temp_df = temp_df.str.replace("\u3000", "\t", regex=False)

temp_df = temp_df.str.replace("/", "", regex=False)


# temp_df = temp_df.str.replace("(", "", regex=False)
# temp_df = temp_df.str.replace(")", "", regex=False)

for i in range(1, 10):
    temp_df = temp_df.str.replace("車" + str(i), "\t車" + str(i), regex=False)
    
for i in range(10):
    temp_df = temp_df.str.replace(str(i)+"分",str(i)+"分\t", regex=False)

temp_df = temp_df.str.replace("\t\t", "\t", regex=False)
temp_df = temp_df.str.replace("駅駅", "駅", regex=False)
temp_df = temp_df.str.replace("線線", "線", regex=False)

for i in range(10):
    temp_df = temp_df.str.replace(str(i)+"分\t",str(i)+"分\t\t", regex=False)


In [508]:
# for i in range(28):
#     print(temp_df.str.split("\t", expand=True)[i].unique())
temp_df_split = temp_df.str.split("\t", expand=True)
# temp_df_split
# temp_df_split[temp_df_split[6] == "(川崎-立川)"]
temp_df_split[3].unique()


array(['徒歩4分', '徒歩5分', '徒歩6分', '徒歩9分', '徒歩2分', '徒歩7分', '徒歩3分', '徒歩8分',
       '徒歩15分', '徒歩11分', '徒歩10分', '徒歩13分', '徒歩12分', '徒歩22分', '徒歩1分',
       '徒歩19分', '徒歩18分', '徒歩23分', '徒歩17分', '徒歩20分', '', '徒歩14分', '徒歩24分',
       '徒歩16分', '徒歩29分', '徒歩37分', '徒歩21分', '徒歩25分', '徒歩27分', '徒歩28分',
       '徒歩33分', '徒歩26分', '徒歩38分', '徒歩36分', '徒歩32分', '徒歩34分', '徒歩39分',
       '車3km(15分', '徒歩30分', '徒歩31分', '徒歩42分', '徒歩40分', '車3.1km(10分',
       '徒歩43分', '車3.9km(11分', '徒歩49分', '徒歩35分', '徒歩44分'], dtype=object)

In [506]:
print(temp_df_split[0][temp_df_split[0].str.contains("線")==False].sum())
print(temp_df_split[1][temp_df_split[1].str.contains("駅")==False].sum())
print(temp_df_split[2][temp_df_split[2].str.contains("バス")==False].sum())
print(temp_df_split[3][temp_df_split[3].str.contains("分")==False].sum())
print(temp_df_split[4][temp_df_split[4].str.contains("下車")==False].sum())

0
0


)武蔵関駅)京成バス)武蔵関駅)深沢小学校前)武蔵関駅)吉祥寺駅)都バス)都バス)吉祥寺駅)都バス)東武バス)新小岩駅)東武バス)都バス)長久保)都バス


In [359]:
total_data["アクセス"].iloc[3]

'総武線・中央線(各停)\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋-荻窪)\t新高円寺駅\t徒歩3分\t\t丸ノ内線(池袋-荻窪)\t東高円寺駅\t徒歩14分'

In [379]:
total_data["アクセス_駅_1"][total_data["アクセス_駅_1"].str.contains("(", regex=False)].unique()

array(['大森(東京都)駅', '金町(東京都)駅', '新富町(東京都)駅', '梅屋敷(東京都)駅', '大塚(東京都)駅',
       '大島(東京都)駅', '中野(東京都)駅', '木場(東京都)駅', '熊野前(舎人ライナー)駅', '田町(東京都)駅',
       '両国(都営線)駅', '神田(東京都)駅', '住吉(東京都)駅', '町屋(東京メトロ)駅', '入谷(東京都)駅',
       '大久保(東京都)駅', 'ときわ台(東京都)駅', '平井(東京都)駅', '稲荷町(東京都)駅', '緑が丘(東京都)駅',
       '森下(東京都)駅', '菊川(東京都)駅', '平和台(東京都)駅', '八丁堀(東京都)駅', '十条(東京都)駅',
       '松原(東京都)駅', '豊島園(西武線)駅', '桜台(東京都)駅', '宝町(東京都)駅', '落合(東京都)駅',
       '大山(東京都)駅', '赤坂(東京都)駅', '早稲田(東京メトロ)駅', '浅草(TX)駅', '三田(東京都)駅',
       '末広町(東京都)駅', '春日(東京都)駅', '長原(東京都)駅', '白山(東京都)駅', '豊島園(都営線)駅',
       '若林(東京都)駅', '高野(東京都)駅', '田原町(東京都)駅', '浦安(千葉県)駅', '早稲田(都電荒川線)駅',
       '京橋(東京都)駅', '大門(東京都)駅', '山下(東京都)駅', '小川町(東京都)駅', '東雲(東京都)駅',
       '牛田(東京都)駅', '日本橋(東京都)駅', '大手町(東京都)駅', '日の出(東京都)駅', '西日暮里(舎人ライナー)駅',
       '栄町(東京都)駅', '向原(東京都)駅', '町屋(京成線)駅'], dtype=object)

In [352]:
total_data["アクセス_線_1"].unique()

array(['都営三田線', '都営大江戸線', '京王線', '総武線・中央線(各停)', '京成金町線', '常磐線快速', '西武池袋線',
       '東急東横線', '南北線', '埼京線', '都営浅草線', '京成本線', '銀座線', '中央線(快速)', '京王新線',
       '東武東上線', '都営新宿線', '半蔵門線', '西武新宿線', '有楽町線', '京浜東北線', 'つくばエクスプレス',
       '小田急小田原線', '副都心線', '東西線', '東急大井町線', '常磐線', '山手線', '東急池上線', '日比谷線',
       '京成押上線', '東武スカイツリーライン', '東急田園都市線', '京急本線', '東急目黒線', '千代田線',
       '丸ノ内線(中野坂上-方南町)', '日暮里・舎人ライナー', '京王井の頭線', '丸ノ内線(池袋-荻窪)', '東武亀戸線',
       '東急多摩川線', 'ゆりかもめ', '都電荒川線', '東急世田谷線', '北総線', '総武本線', '東海道新幹線',
       '京葉線(東京-蘇我)', '高崎線', '西武豊島線', '京急空港線', '横須賀線', 'りんかい線', '西武有楽町線',
       '東武大師線', '東武伊勢崎線(押上-曳舟)', '東北本線(宇都宮線)', '東京モノレール羽田線', '東海道本線',
       '湘南新宿ライン(東北本線-横須賀線)', '湘南新宿ライン(高崎線-東海道本線)', '東北新幹線', '京成成田空港線',
       '中央本線(東京-松本)', '埼玉高速鉄道線'], dtype=object)

In [497]:
total_data["アクセス"][total_data["アクセス"].str.count("\t") == 2]

61                       東急目黒線\t西小山駅\t徒歩10分
72                     常磐線\t金町(東京都)駅\t徒歩18分
151                        東西線\t西葛西駅\t徒歩11分
185                        都営浅草線\t馬込駅\t徒歩1分
219                     京成本線\t堀切菖蒲園駅\t徒歩12分
225                      京王井の頭線\t浜田山駅\t徒歩4分
229                          東西線\t葛西駅\t徒歩7分
236                      西武新宿線\t武蔵関駅\t徒歩15分
287                        都営浅草線\t馬込駅\t徒歩4分
338                         有楽町線\t千川駅\t徒歩2分
350                     東急東横線\t学芸大学駅\t徒歩12分
361                    都営三田線\t板橋区役所前駅\t徒歩7分
403                      東急東横線\t学芸大学駅\t徒歩9分
404                 総武線・中央線(各停)\t小岩駅\t徒歩16分
413                       日比谷線\t南千住駅\t徒歩14分
426                      小田急小田原線\t経堂駅\t徒歩6分
433                総武線・中央線(各停)\t新小岩駅\t徒歩12分
442       西武池袋線\t大泉学園駅\t/バス(7分)大泉風致地区下車徒歩5分
452                       東西線\t門前仲町駅\t徒歩11分
457                          常磐線\t亀有駅\t徒歩4分
464                       都営新宿線\t瑞江駅\t徒歩16分
465                        東急池上線\t池上駅\t徒歩7分
471                     東急世田谷線\t

In [494]:
sorted(total_data["アクセス"].str.count("\t").unique())

[2, 4, 6, 8, 10, 12]